## **구글 드라이브 마운트**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **외부 라이브러리 설치**

In [3]:
# transformer: NLP model
# evaluate: metric for F1 score
!pip install transformers evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 28.4 MB/s 
     |████████████████████████████████| 72 kB 1.3 MB/s 
     |████████████████████████████████| 163 kB 64.8 MB/s 
     |████████████████████████████████| 7.6 MB 56.1 MB/s 
     |████████████████████████████████| 441 kB 64.6 MB/s 
     |████████████████████████████████| 212 kB 74.0 MB/s 
     |████████████████████████████████| 115 kB 77.7 MB/s 
     |████████████████████████████████| 95 kB 4.9 MB/s 
     |████████████████████████████████| 127 kB 75.6 MB/s 
     |████████████████████████████████| 115 kB 75.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      Successfully uninstalled dill-0.3.6


## **Import Libraries**

In [5]:
import os
import json
import torch
import random
import shutil
import evaluate

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
from tqdm.auto import tqdm
from torch.optim import AdamW
from torch.cuda.amp import autocast
from torch.cuda.amp import GradScaler
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split, StratifiedKFold
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, get_scheduler

## **GPU Setting**

In [6]:
# GPU 사용을 위해 cuda 사용이 가능한지 확인합니다.
# 'CPU'라고 나타나는 경우, colab 런타임 유형 변경을 통해 바꾸셔야 합니다. 
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'device:{device}')

device:cuda


## **Hyperparameter Setting**

In [7]:
# model
checkpoint = 'bert-base-uncased'
num_labels = 5
model_path = f'model_with_{checkpoint}'

# training
use_amp = True
num_epochs = 100
lr = 1e-5 # 학습 양상이 처음부터 계속 valid loss 가 증가하는 방향이라 2e-5 에서 낮추게 되었습니다.
batch_size = 32

# scheduler
scheduler_name = 'linear'
num_warmup_steps = 0

# early_stop
early_stop = 3 # 좋아지는 경우가 거의 없어서 GPU 사용량을 효율적으로 운영하고자 값을 줄였습니다.

# seed
seed = 2022

## **Seed Setting**

- 최대한 REPRODUCIBILITY 를 보장하기 위해 제어합니다.
- 단, pytorch 역시 공식적으로 완전히 보장할 수 없다고 말합니다.

- pytorch link: https://pytorch.org/docs/stable/notes/randomness.html

In [8]:
def seed_everything(seed: int = 42, contain_cuda: bool = False):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    print(f"Seed set as {seed}")

In [9]:
seed_everything(seed=2022)

Seed set as 2022


## **Load Original CSV File**

In [10]:
def dataframe_from_csv(target):
      return pd.read_csv(target).rename(columns=lambda x: x.strip())

def dataframe_from_csvs(targets):
  return pd.concat([dataframe_from_csv(x) for x in targets])

In [19]:
data_path = '/content/drive/MyDrive/unidthon/dataset'

train_files = sorted([x for x in Path(f'{data_path}/train/').glob('*.csv')])
val_files = sorted([x for x in Path(f'{data_path}/val/').glob('*.csv')])

train = dataframe_from_csvs(train_files)
val = dataframe_from_csvs(val_files)
test = pd.read_csv(f'{data_path}/test.csv')
print(f'train: {len(train)}')
print(f'validation: {len(val)}')
print(f'test: {len(test)}')

train: 62564
validation: 7820
test: 7820


In [20]:
train['leaktype'].replace(['out','in','noise','other','normal'], [0,1,2,3,4], inplace=True)
val['leaktype'].replace(['out','in','noise','other','normal'], [0,1,2,3,4], inplace=True)
test['leaktype']=""

## **Normalize Dataset**

In [21]:
def normalize(df):
    """
    column 별로 정규화를 시키는 함수입니다.
    이때, 정규화 방식은 표준편차가 아닌 최대 - 최소로 구하였습니다.
    진행한 이유는 특정 column 값이 큰 영향을 끼칠 수 없도록 제한하기 위함입니다.

    Args:
        df (DataFrame): csv 파일을 pandas 로 읽은 데이터

    Returns:
        result (DataFrame) : df 를 column 별로 정규화한 데이터
    """
    result = df.copy()
    
    for feature_name in df.columns:
        # site, sid, leaktype 은 정규화 대상이 아니므로, 값을 그대로 저장하고 넘어갑니다.
        if feature_name in ['site', 'sid', 'leaktype']:
            result[feature_name] = df[feature_name]
            continue
    
        # C01 ~ C26
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value) * 100
        
    return result

In [22]:
def num2alpha(char_dict):
    """
    숫자를 자연어(영어 형태)로 변환하는 함수입니다.
    
    영어를 택한 이유는 크게 2가지입니다.
    - C01 ~ C26 이 26개로 알파벳 개수와 동일하다.
    - 현재 NLP 에서 가장 뛰어난 모델은 대부분 영어 데이터에 최적화되어 있다.
    
    결론부터 말씀드리면 결과는 다음 예시처럼 나옵니다.
    
    예시: 'site: S-4784025026. sid: S-0359369085186035. aa.... zz.'
    
    변환하는 방식은 다음과 같습니다.
    
    1. 각 row 별로 백분율을 구합니다.
    2. 그리고 백분율에 400 을 곱합니다.
    여기서 400 을 곱하는 이유는 'BERT'의 최대 입력 길이를 고려했기 때문입니다.
    최대 길이가 512 인데, site 와 sid 를 제일 앞에 표기한 길이를 대강 112 로 정했습니다.
    그리고 그 후, 나머지 길이 400 을 나머지 row 끼리 나눠서 알파벳을 나열하는 구조입니다.
    3. 이때, 알파벳별로 간격을 두었습니다.

    Args:
        char_dict (Dict): C01 ~ C26 값을 모두 저장한 딕셔너리 자료형

    Returns:
        str : 자연어로 변환된 문자열
    """
    sentence = list()
    
    s = sum(char_dict.values())
    for k, v in char_dict.items():
        a = round(v / s * 400) * chr(96 + int(k[1:]))
        sentence.append(a)

    return ' '.join(sentence)
    

In [23]:
def num2alpha(char_dict):
    """
    숫자를 자연어(영어 형태)로 변환하는 함수입니다.
    
    영어를 택한 이유는 크게 2가지입니다.
    - C01 ~ C26 이 26개로 알파벳 개수와 동일하다.
    - 현재 NLP 에서 가장 뛰어난 모델은 대부분 영어 데이터에 최적화되어 있다.
    
    결론부터 말씀드리면 결과는 다음 예시처럼 나옵니다.
    
    예시: 'site: S-4784025026. sid: S-0359369085186035. aa.... zz.'
    
    변환하는 방식은 다음과 같습니다.
    
    1. 각 row 별로 백분율을 구합니다.
    2. 그리고 백분율에 400 을 곱합니다.
    여기서 400 을 곱하는 이유는 'BERT'의 최대 입력 길이를 고려했기 때문입니다.
    최대 길이가 512 인데, site 와 sid 를 제일 앞에 표기한 길이를 대강 112 로 정했습니다.
    그리고 그 후, 나머지 길이 400 을 나머지 row 끼리 나눠서 알파벳을 나열하는 구조입니다.
    3. 이때, 알파벳별로 간격을 두었습니다.

    Args:
        char_dict (Dict): C01 ~ C26 값을 모두 저장한 딕셔너리 자료형

    Returns:
        str : 자연어로 변환된 문자열
    """
    sentence = list()
    
    s = sum(char_dict.values())
    for k, v in char_dict.items():
        a = round(v / s * 400) * chr(96 + int(k[1:]))
        sentence.append(a)

    return ' '.join(sentence)
    

In [26]:
def tablet2nlp(df):
    """
    : tablet 데이터를 자연어 데이터로 변환하는 함수
    Args:
        df (DataFrame): csv 파일을 읽은 데이터
    Returns:
        DataFrame: 자연어로 변환된 데이터
    """
    nlp_list = list()
    for i, v in df.iterrows():
        nlp_data = list() # 최종적으로 자연어를 저장하는 변수
        char_dict = dict()  # 'C01' ~ 'C26' 을 저장하는 변수
        for c in df.columns:
            if c.startswith('C'):
                char_dict[c] = v[c]
            elif c.startswith('si'): # site, sid
                nlp_data.append(f'{c}: {v[c]}.')
        nlp_data.append(num2alpha(char_dict))
        nlp_list.append(' '.join(nlp_data) + '.')
    df['nlp'] = nlp_list
    # label 과 자연어 데이터 이외에는 모두 삭제합니다.
    df = df[['leaktype', 'nlp']]
    return df

## **Tabular Dataset to Natural Language Dataset**

In [28]:
def data2nlp(df):
    """
    실질적으로 부르는 함수.
    1. 정규화하고,
    2. 자연어로 변환한다.
    
    간단히 표현하고자 있는 함수입니다.
    """
    df = normalize(df)
    return tablet2nlp(df)

In [29]:
# 데이터를 모두 자연어 형태로 변환한다.
nlp_train = data2nlp(train)
nlp_val = data2nlp(val)
nlp_test = data2nlp(test)

In [35]:
# 데이터를 모두 저정해둡니다. 이때, index=False
if not os.path.exists('nlp'):
  os.mkdir('nlp')

nlp_train.to_csv('nlp/train.csv', index=False)
nlp_val.to_csv('nlp/val.csv', index=False)
nlp_test.to_csv('nlp/test.csv', index=False)

## **Load CSV Natural Language file**

In [37]:
data_path = 'nlp'

train_csv = pd.read_csv(os.path.join(data_path, 'train.csv'))
valid_csv = pd.read_csv(os.path.join(data_path, 'val.csv'))
test_csv = pd.read_csv(os.path.join(data_path, 'test.csv'))

## **Customize Dataset**

In [38]:
class UnidthonDataset(Dataset):
  def __init__(self, csv, tokenizer):
    self.dataset = tokenizer(list(csv['nlp']), truncation=True, padding=True, return_tensors='pt')
    self.labels = list(csv['leaktype'])
  
  def __len__(self):
    return len(self.labels)
  
  def __getitem__(self, idx):
    item = {k: torch.tensor(v[idx]) for k, v in self.dataset.items()}
    item['labels'] = torch.tensor(self.labels[idx])

    item = {k: torch.tensor(v[idx]) for k, v in self.dataset.items()}
    if np.isnan(self.labels[idx]): # test dataset 은 값이 비어 있어, NAN 을 방지하기 위해 '0'을 기입합니다.
        item['labels'] = 0
        
    else: 
        item['labels'] = torch.tensor(self.labels[idx])

    return item

## **Load Tokenizer**

In [39]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

## **CSV to DataLoader**

In [40]:
def csv_to_dataloader(csv, tokenizer, mode, batch_size):
  shuffle = True if mode in ['train', 'valid'] else False
  dataset = UnidthonDataset(csv, tokenizer)
  dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

  return dataloader

In [41]:
# dataset
train_dataset = UnidthonDataset(train_csv, tokenizer)
valid_dataset = UnidthonDataset(valid_csv, tokenizer)
test_dataset = UnidthonDataset(test_csv, tokenizer)

In [42]:
# dataloader
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False) # test should not be shuffled !

## **Load Model**

In [43]:
# model
model = AutoModelForSequenceClassification.from_pretrained(checkpoint,
                                                            num_labels=num_labels)
model.to(device)

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

## **Load Optimizer & Scheduler**

In [44]:
# optimizer & scheduler
optimizer = AdamW(model.parameters(), lr=lr)
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(name=scheduler_name, optimizer=optimizer,
                              num_warmup_steps=num_warmup_steps,
                              num_training_steps=num_training_steps)

## **Make Method to infer per epoch**

- 코드의 간결화를 위해 train, eval 모두 하나의 함수에서 가능하도록 만들었습니다.

In [45]:
def inference_per_epoch(tokenizer, model, optimizer, scheduler, use_amp, scaler, dataloader, mode):
  if mode == 'train':
    model.train()
  
  else: # 'valid' or 'test'
    model.eval()

  total_loss = 0
  metric = evaluate.load('f1')

  for batch in tqdm(dataloader, desc=f'{mode} per epoch'):
    batch = { k: v.to(device) for k, v in batch.items() }

    if mode == 'train':
      # colab 인 점을 고려하여, GPU 사용량 및 시간을 절약하고자 사용합니다.
      with torch.cuda.amp.autocast(enabled=use_amp):
        outputs = model(**batch)
        loss = outputs.loss
      
      scaler.scale(loss).backward()
      scaler.step(optimizer)
      scaler.update()

      scheduler.step()
      optimizer.zero_grad()

    else:
      with torch.no_grad():
        outputs = model(**batch)

      loss = outputs.loss

    total_loss += loss.item()
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch['labels'])

  total_loss /= len(dataloader)
  f1 = metric.compute(average='micro')['f1']

  return total_loss, f1

## **Training Loops**

In [46]:
best_model_path = 'best_model'
best_performance = 0
best_epoch = 0
patience = 0 # for Early Stop

scaler = GradScaler(enabled=use_amp)

for epoch in tqdm(range(num_epochs), desc='Training loops'):
  # train per epoch
  train_loss, train_f1 = inference_per_epoch(tokenizer=tokenizer, model=model,
                                              optimizer=optimizer, scheduler=lr_scheduler,
                                              use_amp=use_amp, scaler=scaler,
                                              dataloader=train_dataloader, mode='train')

  # evaluate per epoch
  valid_loss, valid_f1 = inference_per_epoch(tokenizer=tokenizer, model=model,
                                              optimizer=optimizer, scheduler=lr_scheduler,
                                              use_amp=use_amp, scaler=scaler,
                                              dataloader=valid_dataloader, mode='valid')

  print(f'epoch: {epoch}')
  print(f'train_loss: {train_loss}, train_f1: {train_f1}')
  print(f'valid_loss: {valid_loss}, valid_f1: {valid_f1}\n')

  if best_performance < valid_f1:
    patience = 0

    print(f'best performance: {best_performance} → {valid_f1}\n')

    # save best model
    if os.path.exists(best_model_path):
      shutil.rmtree(best_model_path) # delete everything in the directory

    model.save_pretrained(best_model_path)

    best_performance = valid_f1
    best_epoch = epoch

    print(f'saving best model at epoch: {epoch}')
    print(f'best performance: {best_performance} → {valid_f1}\n')

  else:
    patience += 1

  # early_stop
  if early_stop <= patience:
    print('----- Early Stop -----')
    print(f'→ number of patience: {patience}\n')
    break

Training loops:   0%|          | 0/100 [00:00<?, ?it/s]

train per epoch:   0%|          | 0/1956 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]


KeyboardInterrupt: ignored

## **Test the model**

In [47]:
# load best model
best_model = AutoModelForSequenceClassification.from_pretrained(best_model_path, num_labels=num_labels)
best_model.to(device)
best_model.eval()

# inference
answer_list = list()

for batch in test_dataloader:
    batch = { k: v.to(device) for k, v in batch.items() }
    with torch.no_grad():
        outputs = best_model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    answer_list += predictions.detach().cpu().tolist()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]


## **Save the result**

In [48]:
result_csv = pd.DataFrame()
result_csv['site'] = test['site']
result_csv['sid'] = test['sid']
result_csv['leaktype'] = answer_list

result_csv.to_csv('submission.csv', index=False)

In [49]:
result_csv.head()

,site,sid,leaktype
0,S-4784025026,S-0359369085186035,1
1,S-4521010100,S-0359369084102843,4
2,S-2911010107,S-0359369084039755,4
3,S-2711010100,S-0359369084044425,0
4,S-4677025033,S-0359369083955449,0
